Evaluate the performance of different types of optimizer on a LeNet-5 network using MNIST data. At least you need to evaluate SGD, AdaGrad, RMSprop. 

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

In [2]:
import numpy as np
from datetime import datetime 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

import matplotlib.pyplot as plt

# check device
#DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# Preparing for Data
print('==> Preparing data..')

"""
# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

"""

==> Preparing data..


"\n# Training Data augmentation\ntransform_train = transforms.Compose([\n    transforms.RandomCrop(32, padding=4),\n    transforms.RandomHorizontalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n# Testing Data preparation\ntransform_test = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n\n#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')\n\n"

In [4]:
# Preparing for Data
print('==> Preparing data..')

# define transforms
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])


==> Preparing data..


In [5]:
#Defining the convolutional neural network
class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = torch.flatten(out, 1) # flatten all dimensions except batch
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
 
        return out

In [6]:
model1 = LeNet()

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model1.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model1):,} trainable parameters')

The model has 61,706 trainable parameters


In [8]:
########################################################################
# 3. Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

#import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [9]:
def train(model, device, train_loader, optimizer, epoch):
    
    model.train()
    count = 0
    train_loss = 0
    total = 0 
    correct = 0
    
    total_step = len(train_loader)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(device), target.to(device)
        

        #forward pass
        y_pred = model(data)

        loss = criterion(y_pred, target)

        acc = calculate_accuracy(y_pred, target)
        
        
        #Backward pass
        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

In [10]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [12]:
def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 15
    lr = 0.001 # hyper parameter 
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='mnist_data', train=True, download=True, transform=transforms)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='mnist_data', train=False, download=True, transform=transforms)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.RMSprop(model.parameters(), lr=lr, alpha = 0.9, momentum=0.9, weight_decay=5e-4, centered = False, eps=1e-10)
    # alpha - smoothing constant (default: 0.99)
    # eps (float, optional) – term added to the denominator to improve numerical stability (default: 1e-8)
    # momentum (float, optional) – momentum factor (default: 0)
    # lr (float, optional) – learning rate (default: 1e-2)
    # centered (bool, optional) – if True, compute the centered RMSProp, 
    # the gradient is normalized by an estimation of its variance
    # weight_decay (float, optional) – weight decay (L2 penalty) (default: 0)
    

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298496
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.690286
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.541859
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.405471
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.177435
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.193699
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.114021
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.194017
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.170619
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.107012
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.181985
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.128543
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.048190
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.136745
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.304771
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.108032
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.084599
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.048514
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.066825
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.192273
Train Epoch: 1 [

Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.054043
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.026657
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.047042
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.019168
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.196270
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.143460
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.071486
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.093105
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.049046
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.129534
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.120339
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.017155
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.092314
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.044644
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.009849
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.118379
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.026380
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.062237
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.027462
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.062644


Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.125467
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.155470
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.099485
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.032434
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.014306

Test set: Average loss: -4.4511, Accuracy: 9689/10000 (97%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.142439
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.013755
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.057724
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.075795
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.053208
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.008068
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.035183
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.062767
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.087426
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.052219
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.074057
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.147097
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.030665
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.015194
Trai

Train Epoch: 11 [16640/60000 (28%)]	Loss: 0.068811
Train Epoch: 11 [17920/60000 (30%)]	Loss: 0.045052
Train Epoch: 11 [19200/60000 (32%)]	Loss: 0.129173
Train Epoch: 11 [20480/60000 (34%)]	Loss: 0.111960
Train Epoch: 11 [21760/60000 (36%)]	Loss: 0.046053
Train Epoch: 11 [23040/60000 (38%)]	Loss: 0.010355
Train Epoch: 11 [24320/60000 (41%)]	Loss: 0.042093
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.024886
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.008158
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.032868
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.043413
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.052478
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.148045
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.029018
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.104460
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.038646
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.028886
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.028247
Train Epoch: 11 [39680/60000 (66%)]	Loss: 0.017730
Train Epoch: 11 [40960/60000 (6

Train Epoch: 14 [38400/60000 (64%)]	Loss: 0.114813
Train Epoch: 14 [39680/60000 (66%)]	Loss: 0.021408
Train Epoch: 14 [40960/60000 (68%)]	Loss: 0.009653
Train Epoch: 14 [42240/60000 (70%)]	Loss: 0.085002
Train Epoch: 14 [43520/60000 (72%)]	Loss: 0.024058
Train Epoch: 14 [44800/60000 (75%)]	Loss: 0.030839
Train Epoch: 14 [46080/60000 (77%)]	Loss: 0.062075
Train Epoch: 14 [47360/60000 (79%)]	Loss: 0.004640
Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.041050
Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.023500
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.150889
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.117411
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.014414
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.106043
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.081825
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.116988
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.143132

Test set: Average loss: -3.7551, Accuracy: 9811/10000 (98%)

Train Epoch: 15 [0/60000 (0%)]	Loss: 0.006028
Train Epoch: 15 [1280/600

In [13]:
!rm -r mnist_data

'rm' is not recognized as an internal or external command,
operable program or batch file.
